# 在同一设备上配置多个 GitHub 账号的 SSH 密钥

本文档将指导您如何在同一台设备上为多个 GitHub 账号配置不同的 SSH 密钥。

## 1. 理解 SSH 密钥命名

- `id_ed25519` 中的 `ed25519` 是加密算法名称
- Ed25519 是一种现代的椭圆曲线签名算法
- 相比传统的 RSA 算法更安全且性能更好
- `.pub` 后缀的文件是公钥，无后缀的是私钥

## 2. 为每个 GitHub 账号生成独立的 SSH 密钥

为账号1生成SSH密钥：
```bash
ssh-keygen -t ed25519 -C "account1@example.com" -f ~/.ssh/id_ed25519_account1
```

为账号2生成SSH密钥：
```bash
ssh-keygen -t ed25519 -C "account2@example.com" -f ~/.ssh/id_ed25519_account2
```

执行命令时，系统会提示您输入密码（passphrase），可以根据需要选择是否设置。

## 3. 将公钥添加到对应的 GitHub 账号

1. 复制第一个公钥内容：
   ```bash
   type ~/.ssh/id_ed25519_account1.pub
   ```

2. 复制第二个公钥内容：
   ```bash
   type ~/.ssh/id_ed25519_account2.pub
   ```

3. 分别将它们添加到对应的 GitHub 账号设置中：
   - 登录到 GitHub 账号
   - 进入 Settings > SSH and GPG keys
   - 点击 "New SSH key"
   - 将对应的公钥内容粘贴到 Key 字段中
   - 添加标题并保存

## 4. 配置 SSH 客户端

创建或编辑 `~/.ssh/config` 文件，添加以下内容：

```
# GitHub 账号1
Host github-account1
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_ed25519_account1
  IdentitiesOnly yes  # 只使用显式指定的密钥
  IdentityAgent none  # 禁用代理转发

# GitHub 账号2
Host github-account2
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_ed25519_account2
  IdentitiesOnly yes  # 只使用显式指定的密钥
  IdentityAgent none  # 禁用代理转发
```

配置说明：
- `Host`：自定义的主机别名，用于区分不同账号
- `HostName`：真实的目标主机名，这里都是 github.com
- `User`：连接时使用的用户名，GitHub 的 SSH 用户名总是 `git`
- `IdentityFile`：指定该账号使用的私钥文件路径
- `IdentitiesOnly`：设置为 yes 时只会使用显式指定的密钥，避免自动尝试其他密钥
- `IdentityAgent`：设置为 none 时禁用代理转发，确保使用正确的密钥进行认证

这样配置后，您就可以通过不同的 Host 别名来使用不同的密钥。

## 5. 使用不同的账号克隆和推送代码

使用账号1克隆仓库：
```bash
git clone git@github-account1:username1/repository.git
```

使用账号2克隆仓库：
```bash
git clone git@github-account2:username2/repository.git
```

对于已有的仓库，可以修改远程地址：
```bash
# 对于使用账号1的仓库
git remote set-url origin git@github-account1:username1/repository.git

# 对于使用账号2的仓库
git remote set-url origin git@github-account2:username2/repository.git
```

## 6. 测试 SSH 连接

测试账号1的连接：
```bash
ssh -T git@github-account1
```

测试账号2的连接：
```bash
ssh -T git@github-account2
```

如果配置正确，您将看到成功连接的消息。

## 7. 配置第二个GitHub账号

根据您运行 `ssh -T git@github.com` 的结果，您已经成功连接到第一个账号（beckket4141）。现在我们需要为第二个账号设置独立的SSH密钥和配置。

### 7.1 为第二个账号生成新的SSH密钥

如果您还没有为第二个账号生成SSH密钥，请运行以下命令：

```bash
ssh-keygen -t ed25519 -C "second-account@example.com" -f ~/.ssh/id_ed25519_account2
```

### 7.2 将第二个公钥添加到GitHub

1. 显示第二个公钥内容：
   ```bash
   type ~/.ssh/id_ed25519_account2.pub
   ```

2. 复制输出内容，然后登录到您的第二个GitHub账号
3. 进入 Settings > SSH and GPG keys
4. 点击 "New SSH key"
5. 将公钥内容粘贴到 Key 字段中
6. 添加标题（例如："Windows Laptop Second Account"）并保存

### 7.3 更新SSH配置文件

编辑 `~/.ssh/config` 文件，确保包含以下配置：

```
# 第一个GitHub账号（默认）
Host github.com
  HostName ssh.github.com
  Port 443
  User git
  PreferredAuthentications publickey
  IdentityFile ~/.ssh/id_ed25519

# 第二个GitHub账号
Host github-second
  HostName ssh.github.com
  Port 443
  User git
  PreferredAuthentications publickey
  IdentityFile ~/.ssh/id_ed25519_account2
```

### 7.4 测试第二个账号连接

使用以下命令测试第二个账号的连接：

```bash
ssh -T git@github-second
```

如果配置正确，您将看到类似以下的成功消息：
```
Hi second-username! You've successfully authenticated, but GitHub does not provide shell access.
```

## 8. 在项目中使用不同的账号

### 8.1 克隆仓库

使用第一个账号克隆仓库（默认方式）：
```bash
git clone git@github.com:username1/repository.git
```

使用第二个账号克隆仓库：
```bash
git clone git@github-second:username2/repository.git
```

### 8.2 配置仓库的用户信息

对于使用第一个账号的仓库：
```bash
git config user.name "First-Account-Username"
git config user.email "first-account@example.com"
```

对于使用第二个账号的仓库：
```bash
git config user.name "Second-Account-Username"
git config user.email "second-account@example.com"
```

### 8.3 修改现有仓库的远程URL

如果您有一个现有仓库想要使用第二个账号：
```bash
git remote set-url origin git@github-second:username2/repository.git
```

## 9. 故障排除

### 9.1 权限问题

在Windows上，确保SSH目录和文件具有正确的权限：
```bash
icacls ~/.ssh/id_ed25519_second /inheritance:r
icacls ~/.ssh/id_ed25519_second /grant:r "%USERNAME%:F"
```

### 9.2 测试详细输出

如果连接失败，使用详细模式查看问题：
```bash
ssh -Tvvv git@github-second
```

### 9.3 验证配置

检查SSH配置是否正确加载：
```bash
ssh -vT git@github-second
```

### 9.4 检查已加载的密钥

查看SSH代理已加载的密钥：
```bash
ssh-add -l
```